# Example 1: Setting Up Your Archive

This first example runs through the basics of getting started with `MetaViz`. This includes a run-down on the organization of the code, an introduction to the `config.py` file, and instantiating an `Archive` object that you can begin playing with.

More complex interactions with your `Archive` will be covered in Example 2.

## Setting up the `config` file

Before we can begin using the tools and routines available in `MetaViz`, we need to tell it a few details about our collection. The easiest way to do this is by updating the `config.py` file inside the `MetaViz` folder prior to running `setup.py`, which will be read whenever the package is imported. 

Inside the `config` file, you will find the following variables which should be updated to reflect your file collection:

- **`CollectionPath`**: Absolute path to the folder containing your media collection

- **`ExcludeFolders`**: Optional list of folders *inside* `CollectionPath` which are to be excluded from `MetaViz` processing. Every folder named one of the terms in this list will be ignored.

- **`csvPath`**: Absolute path to the folder in which to save intermediate `.csv` files containing the metadata from your collection. This is done for both speed and convenience, as will be explained later.

- **`fields`**: List of XMP/EXIF metadata fields which are of interest in your collection. Usually, files will contain many more metadata fields than are of any real interest most of the time, and this list allows you to refine the metadata you save to the fields you care about. *(Note: If you ever plan to use `MetaViz` to update the metadata in your files, you should be specific when specifying these fields, e.g. writing "XMP-dc:Title" instead of just "Title")*

- **`BackupPath`**: (Optional) Absolute path to the folder in which to save compressed backup files of your media collection. Not required for using the package, but I **strongly recommend** you make sure to have backups of all your files! I've never had an issue with these functions causing trouble, but that doesn't mean it couldn't happen.

Lastly, the `config` file will also loop through `CollectionPath` and store the names of all the subfolders in your collection, and try a bunch of imports to make note of which of the optional packages are available for use.

Whenever the `config` file has been updated, you'll want to run (or re-run) `python setup.py install` to make sure details have been updated in the Python directory.

## Import `MetaViz`

Now that everything has been setup, let's import the `MetaViz` package under the name `mv`. Let's also instantiate an `Archive` object, which will make use of the info stored in the newly-updated `config` file. You can think of this object a bit like a photo album, so we'll give it the name `album`.

In [ ]:
import MetaViz as mv
album = mv.Archive()

Most tools and plotting routines are accessible directly under `mv`, which we will return to later. However, for now, let's focus on getting everything setup for those functions, which we will do via the `album` object.

First, let's check and make sure we've properly read in the details from the updated `config` file. If the `CollectionPath` differs from what was entered earlier, check and make sure things were updated properly.

In [ ]:
print(album.CollectionPath)

## Initializing with ExifTool

*A bit of background*: In order to access the metadata inside our media files, `MetaViz` relies on an installation of the perl package [`exiftool`](https://exiftool.org/), which we interface with via the command line behind the scenes. The way we do this is not particularly sophisticated, but it gets the job done. Because calling `exiftool` to access file metadata is computationally expensive, we only do this once ahead of time, after which we save all of the metadata fields of interest (specified in `config.fields`) inside a folder of `.csv` files (at the location specified in `config.csvPath`). Then, to make use of this metadata in all our functions, we simply use `Pandas` to quickly read all that information into a DataFrame.

Therefore, in order to access the metadata in your files, we first need to utilize `exiftool` and extract a bunch of metadata from your files. We do this by calling `UpdateCSV()`: 

In [ ]:
album.UpdateCSV()

Behind the scenes, what we're doing here is calling exiftool repeatedly with the following bash command:
```
exiftool -csv foldername > csvname
```
which is run for every subfolder inside `CollectionPath`. We break this up by subfolder (rather than doing this recursively with `-r`) because it offers greater flexibility in how we read and update metadata later.

If your collection is unchanging through time, running this function once will be enough. If your collection gets updated through time, this function needs to be re-run to reflect that, at least for the subfolders which are changing. To update only specific subfolders, specify them as a list of strings, i.e. `UpdateCSV(['path/to/subfolder1', 'path/to/subfolder2'])`


## Creating a backup

Before getting carried away with anything else in this package, now seems like another good time to recommend **backing up your files**. For simplicity, we've included a function for doing just that inside the `album` object. 

Here, we will zip up each folder one level below `CollectionPath` and store those backups inside the folder specified in `config.BackupPath`.

*Note: This built-in function may have difficulty compressing files with timestamps older than 1980, which is the beginning of the zip epoch*

In [ ]:
album.CreateBackup()

## Updating metadata

Should the time come that you wish to update the metadata inside your actual media files, we offer a function similar to `UpdateCSV` in reverse: given some modified `.csv` files, the function `UpdateMetadata()` can update the metadata inside your raw files. This again makes use of `exiftool` behind the scenes, using the following bash command:

```
exiftool -csv=csvname foldername -overwrite_original_in_place -P -F
```

For details on these default flags, please see the [`exiftool` FAQ](https://exiftool.org/faq.html). The gist is that, rather than copying your files, we are updating them in place with their existing metadata, and fixing any minor errors should they arise. 

Note that this function accepts a list of specific subfolders to update, which can be much faster (and safer!) than the default of updating every file.

In [ ]:
album.UpdateMetadata()

## Looking forward: Structure of the code

Now that we've finished setting things up and are ready to begin searching and plotting, I think it's a good idea to give a brief overview of the structure of the code. Scripts fall roughly into the following heirarchy, which will show up in later examples:

0. `config` and `tools`: At the base of the package are the config file and several scripts of all-purpose tools, used throughout the rest of the package. 

1. `Archive` class: One step above is these base-level scripts is the `Archive` class we've been using to interface with our collection metadata. Interacting with exiftool and searching through metadata are all achieved by methods inside the `Archive` class.

2. `plot_` scripts: Plotting routines built on top of the previous two levels. These make use of outputs from the previous two levels, but generally don't interface with metadata directly, unless done through an `Archive` object. These are broken into the category of information they display, such as `_timeseries` or `_statistics`

Lastly, we've seperately made a `scripts` folder, which includes some useful non-Python bash scripts for the exiftool commands used behind the scenes here, as well as a few others I've found useful. 